**Importing libraries and Dataset initialization**

In [13]:
!pip install np-utils
!pip install keras

In [15]:
import sklearn.datasets as dataset
import numpy as np
import pandas as pd
import keras
import keras.utils
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle

data= dataset.load_iris()
X= data.data
Y= data.target
Y=Y.reshape(150,1,1)
X=X.reshape(150,1,4)
X= X/np.linalg.norm(X)
x_train,x_test,y_train,y_test=train_test_split(X,Y,train_size=0.33,random_state=42)
y_train= to_categorical(y_train)
y_test= to_categorical(y_test)
print("Size of training dataset size: ", x_train.shape[0], " and testing dataset size: ", y_train.shape[0])

Size of training dataset size:  49  and testing dataset size:  49


In [17]:
class Layer:
    def __init__(self,input_size,output_size):
        self.input=None
        self.output=None
        self.weights= np.random.rand(input_size,output_size)-0.5
        self.biases= np.random.rand(1,output_size)-0.5
    def forward(self,input):
        self.input=input
        self.output= np.dot(self.input,self.weights)+self.biases
        return self.output
    def backward(self,o_error,alpha):
        w_error= np.dot(self.input.T,o_error)
        x_error= np.dot(o_error,self.weights.T)
        self.weights=self.weights + w_error*alpha
        self.biases=self.biases+alpha*o_error
        return x_error

In [18]:
class Activation_class:
    def __init__(self,a_func,act_func_derivative):
        self.activation= a_func
        self.activation_derivative= act_func_derivative
    def forward(self,input):
        self.input= input
        self.output= self.activation(self.input)
        return self.output
    def backward(self,y_error,alpha):
        return self.activation_derivative(self.input)*y_error

In [19]:
class NN:
    def __init__(self):
        self.layers=[]
        self.error= lambda y_pred,y_true: np.mean(np.power(y_true-y_pred,2))
        self.error_derivative= lambda y_true,y_output: 2*(y_true-y_output)/75
    def adding_layer(self,layers):
        self.layers.append(layers)
    def fit_model(self,x,y,itera,alpha):
        for i in range(itera):
            p_e=[]
            net_error=0
            for j in range(49):
                output=x[j]
                for layers in self.layers:
                    output= layers.forward(output)
                error_occured= self.error(output,y[j])
                net_error+=error_occured
                error_derivative= self.error_derivative(y[j],output)
                for layers in reversed(self.layers):
                    error_derivative=layers.backward(error_derivative,alpha,)
            net_error/=49
            if(i%200==0):
                print("iteration: ", i,"error: ",net_error)
        return net_error
    def predict(self,input):
        samples = len(input)
        result =[]
        for i in range(samples):
            output = input[i]
            for layers in self.layers:
                output= layers.forward(output)
            result.append(output)
        return result
    def accuracy(self,input):
        ac = 0
        samples = len(input)
        for i in range(samples):
            y_pred=self.predict(x_test[i])
            y_pred= np.argmax(y_pred)
            if(y_pred == np.argmax(y_test[i])):
                ac+=1
        acc = (ac/samples)*100
        return acc

In [20]:
def Activation_Relu(x):
    output = np.maximum(0, x)
    return output
def Activation_Relu_derivative(x):
     return np.greater(x, 0.).astype(np.float64)

def Activation_softmax_derivative(x):
    return x*(1-x)
def Activation_softmax(input):
    exp_values = np.exp(input.astype(float))
    prob= exp_values/np.sum(exp_values,keepdims=True)
    return prob
def sigmoid(input):
    return 1/(1+(np.exp(-input)))
def sigmoid_derivative(input):
    return sigmoid(input)*(1 - sigmoid(input))

In [29]:
network= NN()
network.adding_layer(Layer(4, 10))
network.adding_layer(Activation_class(Activation_Relu,Activation_Relu_derivative))
network.adding_layer(Layer(10, 8))
network.adding_layer(Activation_class(Activation_Relu,Activation_Relu_derivative))
network.adding_layer(Layer(8, 3))
network.adding_layer(Activation_class(sigmoid,sigmoid_derivative))
network.fit_model(x_train,y_train,30000,0.5)
p=network.predict(x_test)
q= network.accuracy(x_test)

iteration:  0 error:  0.2760278832696755
iteration:  200 error:  0.21835822233548077
iteration:  400 error:  0.21830608776398153
iteration:  600 error:  0.21822589536088086
iteration:  800 error:  0.21808273162957734
iteration:  1000 error:  0.21779320243690997
iteration:  1200 error:  0.21710837796897653
iteration:  1400 error:  0.215063567942995
iteration:  1600 error:  0.2058763782971314
iteration:  1800 error:  0.1581361924945213
iteration:  2000 error:  0.12530317862371412
iteration:  2200 error:  0.1149482879467615
iteration:  2400 error:  0.10913043873146028
iteration:  2600 error:  0.10528477870118665
iteration:  2800 error:  0.10181242704266341
iteration:  3000 error:  0.09809545363125921
iteration:  3200 error:  0.09498323992658869
iteration:  3400 error:  0.09087751987456395
iteration:  3600 error:  0.08437515147965263
iteration:  3800 error:  0.07341204371115737
iteration:  4000 error:  0.059095852730848754
iteration:  4200 error:  0.046624317865987565
iteration:  4400 erro

In [30]:
print(q)

97.02970297029702
